<a href="https://colab.research.google.com/github/mushhub/my-first-blockchain/blob/main/ERC404_mint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ERC404トークン発行のためのGoogle Colaboratoryプログラム
# 必要なライブラリのインストール
!pip install web3 py-solc-x

import json
import time
from web3 import Web3
from solcx import compile_source, install_solc

# Solcコンパイラのインストール
install_solc('0.8.19')

# ERC404コントラクトのSolidityコード
contract_source = """
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.19;

import "@openzeppelin/contracts/token/ERC20/IERC20.sol";
import "@openzeppelin/contracts/token/ERC721/IERC721.sol";
import "@openzeppelin/contracts/token/ERC721/IERC721Receiver.sol";
import "@openzeppelin/contracts/utils/Strings.sol";
import "@openzeppelin/contracts/access/Ownable.sol";

contract ERC404 is IERC20, IERC721, Ownable {
    using Strings for uint256;

    string private _name;
    string private _symbol;
    uint8 private _decimals;
    uint256 private _totalSupply;

    string private _baseURI;

    mapping(address => uint256) private _balances;
    mapping(address => mapping(address => uint256)) private _allowances;

    mapping(uint256 => address) private _owners;
    mapping(address => uint256) private _nftBalances;
    mapping(uint256 => address) private _tokenApprovals;
    mapping(address => mapping(address => bool)) private _operatorApprovals;

    uint256 private _tokenUnit;
    uint256 private _nextTokenId;

    constructor(
        string memory name_,
        string memory symbol_,
        uint8 decimals_,
        uint256 totalSupply_,
        string memory baseURI_
    ) Ownable(msg.sender) {
        _name = name_;
        _symbol = symbol_;
        _decimals = decimals_;
        _baseURI = baseURI_;
        _tokenUnit = 10 ** decimals_;

        _mint(msg.sender, totalSupply_ * _tokenUnit);
    }

    // ERC20機能
    function name() public view returns (string memory) {
        return _name;
    }

    function symbol() public view returns (string memory) {
        return _symbol;
    }

    function decimals() public view returns (uint8) {
        return _decimals;
    }

    function totalSupply() public view override returns (uint256) {
        return _totalSupply;
    }

    function balanceOf(address account) public view override returns (uint256) {
        return _balances[account];
    }

    function transfer(address to, uint256 amount) public override returns (bool) {
        address owner = msg.sender;
        _transfer(owner, to, amount);
        return true;
    }

    function allowance(address owner, address spender) public view override returns (uint256) {
        return _allowances[owner][spender];
    }

    function approve(address spender, uint256 amount) public override returns (bool) {
        address owner = msg.sender;
        _approve(owner, spender, amount);
        return true;
    }

    function transferFrom(address from, address to, uint256 amount) public override returns (bool) {
        address spender = msg.sender;
        _spendAllowance(from, spender, amount);
        _transfer(from, to, amount);
        return true;
    }

    function increaseAllowance(address spender, uint256 addedValue) public returns (bool) {
        address owner = msg.sender;
        _approve(owner, spender, _allowances[owner][spender] + addedValue);
        return true;
    }

    function decreaseAllowance(address spender, uint256 subtractedValue) public returns (bool) {
        address owner = msg.sender;
        uint256 currentAllowance = _allowances[owner][spender];
        require(currentAllowance >= subtractedValue, "ERC404: decreased allowance below zero");
        unchecked {
            _approve(owner, spender, currentAllowance - subtractedValue);
        }
        return true;
    }

    // ERC721機能
    function balanceOfNFT(address owner) public view returns (uint256) {
        return _nftBalances[owner];
    }

    function ownerOf(uint256 tokenId) public view override returns (address) {
        address owner = _owners[tokenId];
        require(owner != address(0), "ERC404: invalid token ID");
        return owner;
    }

    function tokenURI(uint256 tokenId) public view returns (string memory) {
        _requireMinted(tokenId);
        return bytes(_baseURI).length > 0 ? string(abi.encodePacked(_baseURI, tokenId.toString())) : "";
    }

    function approve(address to, uint256 tokenId) public override {
        address owner = ownerOf(tokenId);
        require(to != owner, "ERC404: approval to current owner");
        require(msg.sender == owner || isApprovedForAll(owner, msg.sender),
            "ERC404: approve caller is not token owner or approved for all"
        );
        _tokenApprovals[tokenId] = to;
        emit Approval(owner, to, tokenId);
    }

    function getApproved(uint256 tokenId) public view override returns (address) {
        _requireMinted(tokenId);
        return _tokenApprovals[tokenId];
    }

    function setApprovalForAll(address operator, bool approved) public override {
        _setApprovalForAll(msg.sender, operator, approved);
    }

    function isApprovedForAll(address owner, address operator) public view override returns (bool) {
        return _operatorApprovals[owner][operator];
    }

    function transferFrom(address from, address to, uint256 tokenId) public override {
        require(_isApprovedOrOwner(msg.sender, tokenId), "ERC404: caller is not token owner or approved");
        _transferNFT(from, to, tokenId);
    }

    function safeTransferFrom(address from, address to, uint256 tokenId) public override {
        safeTransferFrom(from, to, tokenId, "");
    }

    function safeTransferFrom(address from, address to, uint256 tokenId, bytes memory data) public override {
        require(_isApprovedOrOwner(msg.sender, tokenId), "ERC404: caller is not token owner or approved");
        _safeTransferNFT(from, to, tokenId, data);
    }

    function supportsInterface(bytes4 interfaceId) public view returns (bool) {
        return
            interfaceId == type(IERC20).interfaceId ||
            interfaceId == type(IERC721).interfaceId;
    }

    // 内部機能
    function _mint(address to, uint256 amount) internal {
        require(to != address(0), "ERC404: mint to the zero address");

        _totalSupply += amount;
        unchecked {
            _balances[to] += amount;
        }
        emit Transfer(address(0), to, amount);

        // NFTの自動発行
        _mintNFTs(to, amount);
    }

    function _mintNFTs(address to, uint256 amount) internal {
        uint256 newNFTs = amount / _tokenUnit;
        for (uint256 i = 0; i < newNFTs; i++) {
            uint256 tokenId = _nextTokenId;
            _nextTokenId++;

            _owners[tokenId] = to;
            unchecked {
                _nftBalances[to] += 1;
            }

            emit Transfer(address(0), to, tokenId);
        }
    }

    function _transfer(address from, address to, uint256 amount) internal {
        require(from != address(0), "ERC404: transfer from the zero address");
        require(to != address(0), "ERC404: transfer to the zero address");

        uint256 fromBalance = _balances[from];
        require(fromBalance >= amount, "ERC404: transfer amount exceeds balance");
        unchecked {
            _balances[from] = fromBalance - amount;
            _balances[to] += amount;
        }

        emit Transfer(from, to, amount);

        // NFTの移動
        _handleNFTTransferForTokens(from, to, amount);
    }

    function _handleNFTTransferForTokens(address from, address to, uint256 amount) internal {
        uint256 tokensPerNFT = _tokenUnit;
        uint256 nftsToTransfer = amount / tokensPerNFT;

        if (nftsToTransfer > 0) {
            uint256 nftCount = 0;
            uint256 maxTokenId = _nextTokenId;

            for (uint256 i = 0; i < maxTokenId && nftCount < nftsToTransfer; i++) {
                if (_owners[i] == from) {
                    _transferNFT(from, to, i);
                    nftCount++;
                }
            }
        }
    }

    function _transferNFT(address from, address to, uint256 tokenId) internal {
        require(ownerOf(tokenId) == from, "ERC404: transfer from incorrect owner");

        delete _tokenApprovals[tokenId];
        unchecked {
            _nftBalances[from] -= 1;
            _nftBalances[to] += 1;
        }
        _owners[tokenId] = to;

        emit Transfer(from, to, tokenId);
    }

    function _safeTransferNFT(address from, address to, uint256 tokenId, bytes memory data) internal {
        _transferNFT(from, to, tokenId);
        require(_checkOnERC721Received(from, to, tokenId, data), "ERC404: transfer to non ERC721Receiver implementer");
    }

    function _approve(address owner, address spender, uint256 amount) internal {
        require(owner != address(0), "ERC404: approve from the zero address");
        require(spender != address(0), "ERC404: approve to the zero address");

        _allowances[owner][spender] = amount;
        emit Approval(owner, spender, amount);
    }

    function _setApprovalForAll(address owner, address operator, bool approved) internal {
        require(owner != operator, "ERC404: approve to caller");
        _operatorApprovals[owner][operator] = approved;
        emit ApprovalForAll(owner, operator, approved);
    }

    function _spendAllowance(address owner, address spender, uint256 amount) internal {
        uint256 currentAllowance = allowance(owner, spender);
        if (currentAllowance != type(uint256).max) {
            require(currentAllowance >= amount, "ERC404: insufficient allowance");
            unchecked {
                _approve(owner, spender, currentAllowance - amount);
            }
        }
    }

    function _isApprovedOrOwner(address spender, uint256 tokenId) internal view returns (bool) {
        address owner = ownerOf(tokenId);
        return (spender == owner || isApprovedForAll(owner, spender) || getApproved(tokenId) == spender);
    }

    function _requireMinted(uint256 tokenId) internal view {
        require(_exists(tokenId), "ERC404: invalid token ID");
    }

    function _exists(uint256 tokenId) internal view returns (bool) {
        return _owners[tokenId] != address(0);
    }

    function _checkOnERC721Received(address from, address to, uint256 tokenId, bytes memory data) private returns (bool) {
        if (to.code.length > 0) {
            try IERC721Receiver(to).onERC721Received(msg.sender, from, tokenId, data) returns (bytes4 retval) {
                return retval == IERC721Receiver.onERC721Received.selector;
            } catch (bytes memory reason) {
                if (reason.length == 0) {
                    revert("ERC404: transfer to non ERC721Receiver implementer");
                } else {
                    assembly {
                        revert(add(32, reason), mload(reason))
                    }
                }
            }
        } else {
            return true;
        }
    }

    // 管理者機能
    function setBaseURI(string memory newBaseURI) public onlyOwner {
        _baseURI = newBaseURI;
    }
}
"""

# コントラクトの設定パラメータ
token_name = "MyERC404Token"
token_symbol = "MET404"
decimals = 18
total_supply = 10000  # トークンの総供給量
base_uri = "https://metadata.example.com/token/"

# テスト用イーサリアムノードの設定（Ganacheを使用する場合）
def deploy_erc404_token():
    # ColabではGanacheをインストールして実行する必要があります
    !npm install -g ganache-cli
    !ganache-cli --deterministic > /dev/null 2>&1 &
    time.sleep(5)  # Ganacheが起動するのを待つ

    # Web3接続の設定
    web3 = Web3(Web3.HTTPProvider('http://127.0.0.1:8545'))

    # アカウントの設定
    private_key = "0x4f3edf983ac636a65a842ce7c78d9aa706d3b113bce9c46f30d7d21715b23b1d"  # Ganacheのデフォルト秘密鍵
    account = web3.eth.account.from_key(private_key)
    web3.eth.default_account = account.address

    print(f"デプロイアカウント: {account.address}")
    print(f"アカウント残高: {web3.eth.get_balance(account.address)} Wei")

    # OpenZeppelinコントラクトのダウンロード（実際のデプロイには必要）
    !mkdir -p node_modules/@openzeppelin/contracts
    !git clone https://github.com/OpenZeppelin/openzeppelin-contracts.git node_modules/@openzeppelin/contracts

    # コントラクトのコンパイル
    compiled_sol = compile_source(
        contract_source,
        output_values=["abi", "bin"],
        solc_version="0.8.19",
        base_path="."
    )

    contract_id, contract_interface = compiled_sol.popitem()
    abi = contract_interface['abi']
    bytecode = contract_interface['bin']

    # コントラクトのデプロイ
    ERC404 = web3.eth.contract(abi